### Here we want to test whether its possible to find an approximate parameter transformation which enables the metric to be flattened

In particular, we want to find a $J$ such that

$$
I \sim J\, g\, J^{\mathrm{T}}
$$

where $g$ is the metric in the original parameter space. We want to do this because we found that for 6D parameter spaces the metric is poorly conditioned, making the calculation of the determinant very unstable. Hopefully we can fix that!

If we decompose the metric as 

$$
g = U^{\mathrm{T}} \, U
$$

then $J = U^{-1}$. Note that once we have found this tranformation we can also transform the parameters as 

$$
\theta_n = \theta_o J
$$

and similarly

$$
\theta_o = \theta_n (J)^{-1}
$$

This is enforced since we want $d\theta_n = d\theta_n g d\theta_n = d\theta_o J g J d\theta_o$

We will start by taking a single point in the parameter space and trying this

In [1]:
from functools import partial
from typing import Callable
import numpy as np
from diffbank.bank import Bank
from diffbank.utils import gen_templates_rejection
from jax import random
import jax
import jax.numpy as jnp
from jax.scipy.linalg import cholesky, inv, det, eigh
from jax import jit
from diffbank.waveforms import kappa6D_modified
from diffbank.waveforms import kappa6D
from diffbank.metric import get_g

In [2]:
f_u = 512.0  # Hz
f_l = 32.0  # Hz

Mt_range = (2, 9)
eta_range = (0.139, 0.25)
chi1_range = (-0.8, 0.8)
chi2_range = (-0.8, 0.8)
k1_range = (0.0, 1.0)
k2_range = (0.0, 1.0)

def get_Sn_aLIGO() -> Callable[[jnp.ndarray], jnp.ndarray]:
    """
    Get interpolator for noise curve.
    """
    xp, yp = np.loadtxt("../scripts/LIGO-P1200087-v18-aLIGO_MID_LOW.txt", unpack=True)
    return lambda f: jnp.interp(f, xp, yp, left=jnp.inf, right=jnp.inf)


def propose(key, n):
    """
    Proposal distribution for var rejection sampling.
    """
    return random.uniform(
        key,
        shape=(
            n,
            6,
        ),
        minval=jnp.stack(
            (Mt_range[0], eta_range[0], chi1_range[0], chi2_range[0], k1_range[0], k2_range[0])
        ),
        maxval=jnp.stack(
            (Mt_range[1], eta_range[1], chi1_range[1], chi2_range[1], k1_range[1], k2_range[1])
        ),
    )

In [3]:
# Lets just check to see if we can get bad metrics
fs = jnp.linspace(f_l, f_u, 3000)
Sn_aLIGO = get_Sn_aLIGO()
mm = 0.95
eta = 0.95

bank = Bank(
    kappa6D.Amp,
    kappa6D.Psi,
    fs,
    Sn_aLIGO,
    sample_base = propose,
    m_star=1 - mm,
    eta=eta,
    name="6D",
)

bank_modified = Bank(
    kappa6D_modified.Amp,
    kappa6D_modified.Psi,
    fs,
    Sn_aLIGO,
    sample_base = propose,
    m_star=1 - mm,
    eta=eta,
    name="6D",
)

seed = 10
key = random.PRNGKey(seed)
N = 100
thetas = propose(key, N)

In [8]:
#This is how we can calculate the appropriate coordinate transform
gs = jax.lax.map(bank.g_fun, thetas)
g_average = gs.sum(axis=0)/gs.shape[0]

# First lets check that the matrix is symmetric and positive definite
def check_symmetric(a, rtol=1e-15, atol=1e-15):
    return np.allclose(a, a.transpose(), rtol=rtol, atol=atol)

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

print(g_average)
print(check_symmetric(np.array(g_average)))
print(is_pos_def(np.array(g_average)))

# Now lets try to take the decomposition
L = cholesky(g_average, lower=True)
J = inv(L)
G = jnp.dot(jnp.dot(J,g_average),J.transpose())
print(G)
print(check_symmetric(np.array(G)))
print(is_pos_def(np.array(G)))

print(J)
np.save("../src/diffbank/waveforms/J_kappa6D.npy", np.array(J))

[[ 1.31221512e+05  1.11958935e+06 -5.08840354e+03 -1.38494552e+03
   7.87588377e+01  9.53182317e+00]
 [ 1.11958935e+06  1.05178032e+07 -5.36831813e+04 -1.36155941e+04
   9.07886059e+02  9.48154277e+01]
 [-5.08840354e+03 -5.36831813e+04  3.61367433e+02  9.39448084e+01
  -6.86153259e+00 -7.41476228e-01]
 [-1.38494552e+03 -1.36155941e+04  9.39448084e+01  3.08670897e+01
  -1.73461442e+00 -2.84331164e-01]
 [ 7.87588377e+01  9.07886059e+02 -6.86153259e+00 -1.73461442e+00
   2.49597439e-01  1.53831056e-02]
 [ 9.53182317e+00  9.48154277e+01 -7.41476228e-01 -2.84331164e-01
   1.53831056e-02  5.60851606e-03]]
True
True
[[ 1.00000000e+00 -4.44089210e-16  6.66133815e-16  2.22044605e-16
   1.11022302e-16  4.44089210e-16]
 [ 0.00000000e+00  1.00000000e+00 -2.66453526e-15  2.88657986e-15
   1.11022302e-16  4.99600361e-16]
 [ 3.13839547e-16 -2.83850511e-15  1.00000000e+00  3.10862447e-15
  -4.44089210e-16  4.44089210e-16]
 [ 3.13839547e-16  2.71538090e-15  4.42784677e-15  1.00000000e+00
   1.34614542e

In [5]:
k = 0
print(gs[k])
print(jnp.dot(jnp.dot(J,gs[k]),J.T))
print(jnp.sqrt(det(jnp.dot(jnp.dot(J,gs[k]),J.T))))

[[ 1.42557956e+04  3.14955033e+05 -3.24663204e+03 -4.97371980e+02
   6.09051870e+01  2.00263795e-01]
 [ 3.14955033e+05  6.96120514e+06 -7.13727604e+04 -1.09358693e+04
   1.33675510e+03  4.39561489e+00]
 [-3.24663204e+03 -7.13727604e+04  7.83503795e+02  1.19804031e+02
  -1.49656904e+01 -4.91844246e-02]
 [-4.97371980e+02 -1.09358693e+04  1.19804031e+02  1.83202001e+01
  -2.28699358e+00 -7.51627567e-03]
 [ 6.09051870e+01  1.33675510e+03 -1.49656904e+01 -2.28699358e+00
   2.87462515e-01  9.44598523e-04]
 [ 2.00263795e-01  4.39561489e+00 -4.91844246e-02 -7.51627567e-03
   9.44598523e-04  3.10395251e-06]]
[[ 0.10863917  0.54316065 -0.23767474  0.32689749 -0.09772111 -0.11689447]
 [ 0.54316065  2.71859515 -1.13524838  1.59219122 -0.48770107 -0.58064566]
 [-0.23767474 -1.13524838  1.46832939 -1.46936072  0.22968633  0.32348672]
 [ 0.32689749  1.59219122 -1.46936072  1.58344371 -0.30657127 -0.40561096]
 [-0.09772111 -0.48770107  0.22968633 -0.30657127  0.08878457  0.10631948]
 [-0.11689447 -0.5

In [6]:
# Here is a demonstration of the problem
# For some points in the parameter space, the density is nan
# This is due to a poorly conditioned metric
for i in range(0, 10):
    print(bank.density_fun(thetas[i]))

1.231819280168623e-13
nan
nan
2.427226962213465e-13
7.885662540237092e-14
2.4650504553996393e-15
nan
2.5210887163890613e-15
1.1969542749845183e-13
1.664126844623099e-14


In [9]:
# This should no longer get nans
for i in range(0, 10):
    theta_n = jnp.dot(inv(J), thetas[i])
    print("Original: %.15f" % jnp.sqrt(det(jnp.dot(jnp.dot(J,gs[i]),J.T))))
    print("Modified: %.15f" % jnp.sqrt(det(bank_modified.g_fun(theta_n))))

Original: 0.000000000000000
Modified: nan
Original: nan
Modified: nan
Original: nan
Modified: 0.000000000000000
Original: 0.000000000000000
Modified: nan
Original: 0.000000000000000
Modified: 0.000000000000000
Original: 0.000000000000000
Modified: 0.000000000000000
Original: nan
Modified: nan
Original: 0.000000000000000
Modified: 0.000000000000000
Original: 0.000000000000000
Modified: 0.000000000000000
Original: 0.000000000000000
Modified: 0.000000000000000
